# Librerias

In [1]:
import pandas as pd
import pyodbc
from tqdm import tqdm 
from datetime import date, datetime
import unicodedata
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.utils import compute_class_weight
import joblib
import matplotlib
import matplotlib.pyplot as plt
import shap

#### Base estudiantes

In [2]:
# Conexión
server = '172.16.1.33'
database = 'CUN_REPOSITORIO'

try:
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )

    # Tabla principal
    query_inicial = """
WITH Ultima_Liquidacion AS (
    SELECT 
        Identificacion,
        MAX(Fecha_Liquidacion) AS Ultima_Fecha
    FROM Desercion.Resultado_Final_CUN
    WHERE Fecha_Liquidacion < '2025-07-01'
    GROUP BY Identificacion
)
SELECT 
    rf.Identificacion AS Identificacion,
    rf.Periodo AS Periodo,
    rf.Status AS DescRF_Status,
    rf.Tipo_estado_alumno AS DescRF_Tipo_estado_alumno,
    rf.Modalidad AS DescRF_Modalidad,
    rf.Semestre_SINU AS DescRF_Semestre_SINU,
    rf.ciclo AS DescRF_ciclo,
    rf.SEMESTRE_MEN AS DescRF_SEMESTRE_MEN,
    rf.Genero AS DescRF_Genero,
    rf.Unidad AS DescRF_Unidad,
    rf.Jornada AS DescRF_Jornada,
    rf.Periodo_Ult_Pago AS DescRF_Periodo_Ult_Pago,
    rf.Sede AS DescRF_Sede,
    rf.Regional AS DescRF_Regional,
    rf.Programa AS DescRF_Programa,
    rf.Acceso_Moodle AS DescRF_Acceso_Moodle,
    rf.tipo_estudio AS DescRF_tipo_estudio,
    rf.Id_Alumn_Programa AS Id_Alumn_Programa,
    rf.Tipo_Alumno AS DescRF_Tipo_Alumno,
    rf.Nuevo AS DescRF_Nuevo,
    rf.orden AS DescRF_orden,
    rf.Fecha_Liquidacion AS DescRF_Fecha_Liquidacion,
    rf.Fecha_Nacimiento AS DescRF_Fecha_Nacimiento,
    rf.ultimo_curso AS DescRF_ultimo_curso,
    dm.[MATERIAS INSCRITAS] AS DescAM_MATERIAS_INSCRITAS,
    dm.[MATERIAS APROBADAS] AS DescAM_MATERIAS_APROBADAS,
    dm.[Porcentaje_aprobacion] AS DescAM_Porcentaje_aprobacion,
    ee.promedio as EEpromedio,
    ee.UNIDADNEGOCIO AS EE_UNIDADNEGOCIO,
    ee.DEPARTAMENTO_REGIONAL AS EE_DEPARTAMENTO_REGIONAL,
    ee.OTRA_DISCAPACIDAD AS EE_OTRA_DISCAPACIDAD,
    ee.SISTEMA_SALUD AS EE_SISTEMA_SALUD,
    ee.ESTRATO_ACTUALIZADO AS EE_ESTRATO_ACTUALIZADO,
    ee.EMPRESA AS EE_EMPRESA
FROM Desercion.Resultado_Final_CUN rf
JOIN Ultima_Liquidacion ul 
    ON rf.Identificacion = ul.Identificacion 
   AND rf.Fecha_Liquidacion = ul.Ultima_Fecha
JOIN Desercion.PORCENTAJE_APROBACION_MATERIAS dm  
    ON rf.Identificacion = dm.IDENTIFICACION 
   AND rf.Periodo = dm.COD_PERIODO
LEFT JOIN CUN.ESTADISTICA_ESTUDIANTE_2 AS ee  
    ON rf.Identificacion = ee.NUM_IDENTIFICACION 
   AND rf.Periodo = ee.COD_PERIODO
LEFT JOIN Financiera.Recibos_caja AS rc 
    ON rf.Identificacion = CONVERT(VARCHAR(20), rc.cliente) 
   AND rc.periodo = rf.Periodo
LEFT JOIN CUN.Zoho_Sinu AS zz  
    ON rf.Identificacion = zz.NUM_IDENTIFICACION
LEFT JOIN Financiera.Recibos_Caja_Medios_Pago AS mp 
    ON rf.Identificacion = mp.IDENTIFICACION 
   AND rf.periodo = mp.PERIODO;
      """
    df_inicial = pd.read_sql(query_inicial, conn)

    # Cerrar conexión 
    conn.close()

except Exception as e:
    print("Error:", e)


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\2106066819.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inicial = pd.read_sql(query_inicial, conn)


In [3]:
df_inicial

,Identificacion,Periodo,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_SEMESTRE_MEN,DescRF_Genero,DescRF_Unidad,...,DescAM_MATERIAS_INSCRITAS,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EEpromedio,EE_UNIDADNEGOCIO,EE_DEPARTAMENTO_REGIONAL,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA
0,00010105021,2017B,Desercion CUN,NUEVO,Presencial,1,TECNICO,2,M,Técnica Profesional en Logística de Comercio E...,...,10,8,80.00,None,None,None,None,None,None,None
1,00010308259,2018A,Desercion CUN,ANTIGUO,Presencial,2,TECNICO,1,F,Técnica Profesional en Logística de Comercio E...,...,7,7,100.00,None,None,None,None,None,None,None
2,00010606926,2017A,Desercion CUN,NUEVO,Presencial,1,TECNICO,1,M,TÉCNICA PROFESIONAL EN SERVICIOS TURÍSTICOS Y ...,...,7,4,57.14,None,None,None,None,None,None,None
3,00011104070,2018A,Desercion CUN,ANTIGUO,Presencial,2,TECNICO,1,F,TECNICA PROFESIONAL EN PROCESOS GRAFICOS,...,9,1,11.11,None,None,None,None,None,None,None
4,00011206301,2017B,Desercion CUN,NUEVO,Distancia,1,TECNICO,2,M,TECNICA PROFESIONAL EN PROCESOS ADMINISTRATIVOS,...,9,8,88.89,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176908,9976845,2025A,Rematriculado,ANTIGUO,Presencial,6,PROFESIONAL,1,M,TÉCNICA PROFESIONAL EN SERVICIOS ADMINISTRATIV...,...,7,0,0.00,None,None,None,None,None,None,None
176909,9976918,2017A,Graduado,ANTIGUO,Distancia,10,PROFESIONAL,1,M,Técnica Profesional en Contabilidad y Finanzas,...,7,7,100.00,None,None,None,None,None,None,None
176910,9977464,22V01,Desercion CUN,Nuevo - Homologado,Virtual,2,TECNICO,1,M,TÉCNICA PROFESIONAL EN REGISTRO DE PROCESOS PR...,...,8,8,100.00,None,None,None,None,None,None,None
176911,9977702,21V06,Retiro,Nuevo - Homologado,Virtual,7,PROFESIONAL,2,M,TÉCNICA PROFESIONAL EN ADMINISTRACIÓN DE PROCE...,...,7,0,0.00,None,None,None,None,None,None,None


In [ ]:
df_final_final['Tipo_estado_alumno'].value_counts()

#### Notas

In [4]:
# Conexión
server = '172.16.1.33'
database = 'CUN_REPOSITORIO'

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Lotes
ids_periodos = df_inicial[['Id_Alumn_Programa', 'Periodo']].drop_duplicates()

df_notas_total = pd.DataFrame()

# Iterar por periodo
for periodo, subdf in tqdm(ids_periodos.groupby('Periodo'), desc="Descargando por periodo"):
    ids = subdf['Id_Alumn_Programa'].astype(str).tolist()

    # dividir en lotes para que el IN no sea demasiado largo
    chunk_size = 500  
    for i in range(0, len(ids), chunk_size):
        ids_chunk = ids[i:i+chunk_size]

    
        ids_escaped = [id_.replace("'", "''''") for id_ in ids_chunk]
        ids_str = ",".join([f"''{id_}''" for id_ in ids_escaped])

        query_chunk = f"""
        SELECT *
        FROM OPENQUERY([172.16.1.175], '
            SELECT 
                ID_ALUM_PROGRAMA,
                COD_MATERIA,
                COD_PERIODO,
                DEF_HISTORIA,
                NOT_PERIODO
            FROM SRC_HIS_ACADEMICA
            WHERE COD_PERIODO=''{periodo}''
              AND ID_ALUM_PROGRAMA IN ({ids_str})
        ') AS n
        """

        df_chunk = pd.read_sql(query_chunk, conn)
        df_notas_total = pd.concat([df_notas_total, df_chunk], ignore_index=True)

# Cerrar conexión
conn.close()


Descargando por periodo:   0%|          | 0/101 [00:00<?, ?it/s]C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\470199164.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql(query_chunk, conn)
Descargando por periodo:   1%|          | 1/101 [00:01<02:24,  1.45s/it]C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\470199164.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql(query_chunk, conn)
Descargando por periodo:   2%|▏         | 2/101 [00:01<01:07,  1.46it/s]C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\470199164.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or 

#### Asistencia

In [ ]:
# # Conexión
# conn = pyodbc.connect(
#     'DRIVER={ODBC Driver 17 for SQL Server};'
#     f'SERVER={server};'
#     f'DATABASE={database};'
#     'Trusted_Connection=yes;'
# )

# # Tabla notas
# query_asistencia = """
#     SELECT 
#         AESPA_IDENTIFICACION,
#         PERIODO,
#         AESPA_COD_MATERIA,
#         Nom_materia,
#         AESPA_ASISTENCIA
        
#     FROM [dbo].[vw_AESPA_ASISTENCIA_ESTUDTS_SPA]
# """
# df_asistencia = pd.read_sql(query_asistencia, conn)

# # Cerrar conexión 
# conn.close()


#### Inicio clase

In [5]:
# Crear conexión
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Consulta para obtener esas columnas
query_inicio_clases = """
SELECT 
    num_identificacion,
    COD_PERIODO,
    ESTADO_PAGO,
    EstadoMoodle,
    Ultimo_Acc_Mood,
    Correo,
    Horario
    
FROM dbo.dwh_Inicio_clases
"""
df_inicio_clases = pd.read_sql(query_inicio_clases, conn)

# Cerrar conexión
conn.close()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\3570879745.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inicio_clases = pd.read_sql(query_inicio_clases, conn)


#### Financiacion

In [6]:
#LLAVE NUMERO_DOCUMENTO_ESTUDIANTE, PERIODO
# Conexión
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Tabla notas
query_financiaciones = """
SELECT 
    NUMERO_DOCUMENTO_ESTUDIANTE,
    VALOR_MATRICULA,
    VALOR_FINANCIACION,
    VALOR_TOTAL_FINANCIACION,
    VALOR_CUOTA_INICIAL,
    CUOTAS,
    VALOR_CUOTA,
    PERIODO
FROM Financiera.FINANCIACIONES_CT_AYUDA
"""
df_financiaciones = pd.read_sql(query_financiaciones, conn)


# Cerrar conexión 
conn.close()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\556862819.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_financiaciones = pd.read_sql(query_financiaciones, conn)


In [7]:

# Conexión
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Tabla notas
query_financiaciones2 = """
SELECT 
    IDENTIFICACION,
    PERIODO,
    VALOR_FINANCIACION
FROM Financiera.CTAYUDA
"""
df_financiaciones2 = pd.read_sql(query_financiaciones2, conn)


# Cerrar conexión 
conn.close()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\2794303963.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_financiaciones2 = pd.read_sql(query_financiaciones2, conn)


### Financiera

In [8]:
#LLAVE NUMERO_DOCUMENTO_ESTUDIANTE, PERIODO
# Conexión
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Tabla notas
query_financiera = """
SELECT *
FROM Financiera.Recibos_Caja_Medios_Pago
"""
df_financiera = pd.read_sql(query_financiera, conn)


# Cerrar conexión 
conn.close()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\992456918.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_financiera = pd.read_sql(query_financiera, conn)


#### Información inducción

In [ ]:
#df_induccion = pd.read_excel("C:/Users/maria_castrob/OneDrive - Corporación Unificada Nacional de Educación Superior - CUN/Permanencia/ASISTENCIA INDUCCIONES.xlsx")

In [ ]:
# df_induccion = df_induccion.rename(columns={
#     'Número de documento de identidad:': "Numero_documento"
# })


#### Tickets

In [9]:
#LLAVE NUMERO_DOCUMENTO_ESTUDIANTE, PERIODO
# Conexión
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Tabla notas
query_tickets = """
SELECT 
    NumeroDocumento,
    Periodo,
    COUNT(*) AS Total_Tickets
FROM [ZOHO].[dbo].[VW_Tickets]
GROUP BY NumeroDocumento, Periodo
"""
df_tickets = pd.read_sql(query_tickets, conn)


# Cerrar conexión 
conn.close()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39324\68571508.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tickets = pd.read_sql(query_tickets, conn)


#### Cruzar la base, con las notas del ultimo periodo del estudiante

In [10]:
df_final_notas = pd.merge(
    df_inicial,
    df_notas_total,
    left_on=['Id_Alumn_Programa', 'Periodo'],
    right_on=['ID_ALUM_PROGRAMA', 'COD_PERIODO'],
    how='left'
)


#### Cruzar base con la asistencia por materia del último periodo, agrupado por materia

In [ ]:
# df_final_asistencia = pd.merge(
#     df_final_notas,
#     df_asistencia,
#     left_on=['Identificacion', 'Periodo', 'COD_MATERIA'],
#     right_on=['AESPA_IDENTIFICACION', 'PERIODO', 'AESPA_COD_MATERIA'],
#     how='left'
# )



#### Unión base con info de inicio de clase

In [11]:
df_final_inicio_clase = pd.merge(
    df_final_notas,
    df_inicio_clases,
    left_on=['Identificacion', 'Periodo'],
    right_on=['num_identificacion','COD_PERIODO'],
    how='left'
)


#### Unión base financiación

In [12]:
df_final_inicio_clase['Identificacion'] = df_final_inicio_clase['Identificacion'].astype(str)
df_financiaciones['NUMERO_DOCUMENTO_ESTUDIANTE'] = df_financiaciones['NUMERO_DOCUMENTO_ESTUDIANTE'].astype(str)

df_final_inicio_clase['Periodo'] = df_final_inicio_clase['Periodo'].astype(str)
df_financiaciones['PERIODO'] = df_financiaciones['PERIODO'].astype(str)

df_final_financiaciones = pd.merge(
    df_final_inicio_clase,
    df_financiaciones2,
    left_on=['Identificacion', 'Periodo'],
    right_on=['IDENTIFICACION','PERIODO'],
    how='left'
)


In [13]:
df_final_financiaciones['VALOR_FINANCIACION'].value_counts()

VALOR_FINANCIACION
852570.0     1807
1155251.0    1202
850729.0     1173
1156010.0    1134
1041485.0    1122
             ... 
413953.0        1
253766.0        1
329040.0        1
681575.0        1
1405860.0       1
Name: count, Length: 6144, dtype: int64

#### Unión base financiera

In [14]:
df_final_financiera = pd.merge(
    df_final_financiaciones,
    df_financiera,
    left_on=['Identificacion', 'Periodo'],
    right_on=['IDENTIFICACION','PERIODO'],
    how='left'
)

### Unión info inducción

In [ ]:
# df_final_induccion = pd.merge(
#     df_final_financiera,
#     df_induccion[["Numero_documento"]].drop_duplicates(),
#     left_on="Identificacion",
#     right_on="Numero_documento",
#     how="left"
# )

# # Columna 1/0
# df_final_induccion["Asistencia"] = df_final_induccion["Numero_documento"].notna().astype(int)

# # Eliminar auxiliar
# df_final_induccion = df_final_induccion.drop(columns=["Numero_documento"])


#### Unión base tickets

In [15]:
df_final_final = pd.merge(
    df_final_financiera,
    df_tickets,
    left_on=['Identificacion', 'Periodo'],
    right_on=['NumeroDocumento','Periodo'],
    how='left'
)

In [16]:
columnas_a_eliminar = [
    'ID_ALUM_PROGRAMA',
    'COD_PERIODO_x',
    'PERIODO_x',
    'AESPA_COD_MATERIA',
    'num_identificacion',
    'COD_PERIODO_y',
    'NUMERO_DOCUMENTO_ESTUDIANTE',
    'PERIODO_y',
    'IDENTIFICACION',
    'PERIODO',
    'NumeroDocumento',
    'IDENTIFICACION_x',
    'IDENTIFICACION_y'
]

df_final_final = df_final_final.drop(columns=columnas_a_eliminar, errors="ignore")


## Limpieza de los datos

In [17]:
#Crear edad
df_final_final['DescRF_Fecha_Nacimiento'] = pd.to_datetime(df_final_final['DescRF_Fecha_Nacimiento'])

# Función para calcular edad
def calcular_edad(fecha_nacimiento):
    hoy = date.today()
    return hoy.year - fecha_nacimiento.year - (
        (hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day)
    )

df_final_final['Edad'] = df_final_final['DescRF_Fecha_Nacimiento'].apply(lambda x: calcular_edad(x.date()))


In [18]:
# Status trabajo
df_final_final['Trabaja'] = df_final_final['EE_EMPRESA'].apply(lambda x: 'TRABAJA' if pd.notnull(x) and str(x).strip() != '' else 'NO TRABAJA')

In [19]:
# Estandarizar unidad
def estandarizar(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).upper()  
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8') 
    texto = ' '.join(texto.split())  
    return texto

df_final_final['DescRF_Unidad'] = df_final_final['DescRF_Unidad'].apply(estandarizar)
df_final_final['DescRF_Unidad'] = df_final_final['DescRF_Unidad'].replace({
    'ADMINISTRACION EMPRESAS AGROINDUSTRIALES': 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES',
    'DISENO DE MODA': 'PROFESIONAL EN DISENO DE MODAS',
    'DISENO DE MODAS': 'PROFESIONAL EN DISENO DE MODAS',
    'TECNOLOGIA EN GESTION DE PROCESOS AGROINSDUSTRIALES':'TECNOLOGIA EN GESTION DE PROCESOS AGROINDUSTRIALES',
    'ADMINISTRACION DE EMPRESAS': 'PROFESIONAL EN ADMINISTRACION DE EMPRESAS',
    'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES':'PROFESIONAL EN ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES',
    'ADMINISTRACION DE LA SEGURIDAD SOCIAL': 'PROFESIONAL EN ADMINISTRACION DE LA SEGURIDAD SOCIAL',
    'ADMINISTRACION DE SERVICIOS DE SALUD': 'PROFESIONAL EN ADMINISTRACION DE SERVICIOS DE SALUD',
    'ADMINISTRACION DEPORTIVA - BOGOTA': 'PROFESIONAL EN PROCESOS DE ADMINISTRACION DEPORTIVA - BOGOTA',
    'ADMINISTRACION PUBLICA': 'PROFESIONAL EN ADMINISTRACION PUBLICA',
    'ADMINISTRACION TURISTICA Y HOTELERA': 'PROFESIONAL EN ADMINISTRACION TURISTICA Y HOTELERA',
    'COMUNICACION SOCIAL': 'PROFESIONAL EN COMUNICACION SOCIAL',
    'CONTADURIA PUBLICA': 'PROFESIONAL EN CONTADURIA PUBLICA',
    'PROFESIONAL EN CONTADURIA CONTABLE': 'PROFESIONAL EN CONTADURIA PUBLICA',
    'DERECHO': 'PROFESIONAL EN DERECHO',
    'DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES': 'PROFESIONAL EN DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES',
    'DISENO GRAFICO': 'PROFESIONAL EN DISENO GRAFICO',
    'DISENO Y PRODUCCION DE MODA': 'PROFESIONAL EN DISENO Y PRODUCCION DE MODA',
    'INGENIERIAS DE SISTEMAS': 'PROFESIONAL EN INGENIERIA DE SISTEMAS',
    'INGENIERIA DE SISTEMAS': 'PROFESIONAL EN INGENIERIA DE SISTEMAS',
    'INGENIERIA DE ELECTRONICA': 'PROFESIONAL EN INGENIERIA ELECTRONICA',
    'INGENIERIA ELECTRONICA': 'PROFESIONAL EN INGENIERIA ELECTRONICA',
    'INGENIERIA DE INDUSTRIAL': 'PROFESIONAL EN INGENIERIA INDUSTRIAL',
    'NEGOCIOS INTERNACIONALES': 'PROFESIONAL EN NEGOCIOS INTERNACIONALES',
    'TECNICA PROFESIONAL EN SOPORTE DE SISTEMAS EN INFORMATICA': 'TECNICA PROFESIONAL EN SOPORTE DE SISTEMAS E INFORMATICA',
    'PROFESIONAL EN COMUNICACION SOCIAL - PERIODISMO': 'PROFESIONAL EN COMUNICACION SOCIAL',
    'TECNICA PROFESIONAL EN CONTABILIDAD Y FINANZAS': 'TECNICA PROFESIONAL EN CONTABILIDAD',
    'TECNICA PROFESIONAL EN DISENO DE MODA Y PATRONAJE': 'TECNICA PROFESIONAL EN DISENO DE VESTUARIO Y PATRONAJE',
    'TECNICA PROFESIONAL EN OPERACION DE PROCESOS DE PRODUCCION GRAFICA- CHILLERSITARIO': 'TECNICA PROFESIONAL EN OPERACION DE PROCESOS DE PRODUCCION GRAFICA',
    'TECNOLOGIA EN DESARROLLO DE SOFTWARE Y REDES': 'TECNOLOGIA EN DESARROLLO DE SOFTWARE',
    'TECNOLOGIA EN GESTION CONTABLE FINANCIERA': 'TECNOLOGIA EN GESTION CONTABLE Y FINANCIERA',
    'TECNICO PROFESIONAL EN PRODUCCION DE CONTENIDOS INFORMATIVOS': 'TECNICA PROFESIONAL EN PRODUCCION DE CONTENIDOS INFORMATIVOS',
    'TECNICO PROFESIONAL EN PROCESOS CONTABLES': 'TECNICA PROFESIONAL EN PROCESOS CONTABLES',
    'TECNICO PROFESIONAL EN PROCESOS ADMINISTRATIVOS': 'TECNICA PROFESIONAL EN PROCESOS ADMINISTRATIVOS',
    'TECNICO PROFESIONAL EN PERIODISMO INFORMATIVO': 'TECNICA PROFESIONAL EN PERIODISMO INFORMATIVO',
    'TECNICO PROFESIONAL EN DISENO DIGITAL': 'TECNICA PROFESIONAL EN DISENO DIGITAL',
    'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES': 'PROFESIONAL EN ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES',
    'CONTADURIA PUBLICA - VIRTUAL': 'PROFESIONAL EN CONTADURIA PUBLICA - VIRTUAL',
    'INGENIERIA DE SISTEMAS-VIRTUAL':'PROFESIONAL EN INGENIERIA DE SISTEMAS-VIRTUAL',
    'INGENIERIA ELECTRONICA': 'PROFESIONAL EN INGENIERIA ELECTRONICA',
    'INGENIERIA INDUSTRIAL': 'PROFESIONAL EN INGENIERIA INDUSTRIAL',
    'PUBLICIDAD Y MERCADEO': 'PROFESIONAL EN PUBLICIDAD Y MERCADEO'
})

In [20]:
# Estandarizar programa
def estandarizar(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).upper()  
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8') 
    texto = ' '.join(texto.split())  
    return texto

df_final_final['DescRF_Programa'] = df_final_final['DescRF_Programa'].apply(estandarizar)
df_final_final['DescRF_Programa'] = df_final_final['DescRF_Programa'].replace({
    'COMUNICACION SOCIAL Y PERIODISMO': 'COMUNICACION SOCIAL',
    'ADMINISTRACION EN SERVICIOS DE SALUD': 'ADMINISRACION DE SERVICIOS DE SALUD'
})

In [21]:
# Estandarizar regional
def estandarizar(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).upper() 
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')  
    texto = ' '.join(texto.split())  
    return texto

df_final_final['EE_DEPARTAMENTO_REGIONAL'] = df_final_final['EE_DEPARTAMENTO_REGIONAL'].apply(estandarizar)

In [22]:
# Estandarizar discapacidad
def estandarizar(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).upper() 
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')  
    texto = ' '.join(texto.split()) 
    return texto

df_final_final['EE_OTRA_DISCAPACIDAD'] = df_final_final['EE_OTRA_DISCAPACIDAD'].apply(estandarizar)
df_final_final['EE_OTRA_DISCAPACIDAD'] = df_final_final['EE_OTRA_DISCAPACIDAD'].replace({
    'NINGUNO': 'NINGUNA',
    'DISCAPACIDAD FISICA': 'FISICA'
})    

In [23]:
# Clasificar las deserciones
def target(x): 
    if x == 'Desercion CUN':
        return 1 
    else: 
        return 0 
df_final_final['DescRF_Status'] = df_final_final.apply(lambda x: target(x.DescRF_Status),axis=1)

In [24]:
# Estandarizar estados
def rec_tipo_estado(x): 
    if x == 'Nuevo - Homologado':
        return 'NUEVO'
    else: 
        return x 
df_final_final['DescRF_Tipo_estado_alumno'] = df_final_final.apply(lambda x: rec_tipo_estado(x.DescRF_Tipo_estado_alumno),axis=1)

In [25]:
# Estandarizar modalidad
def rec_modalidad(x):
    if x == 'Presencial Fin De Semana':
        return 'Presencial'
    elif x == 'No Presencial':
        return 'Virtual'
    else:
        return x 
df_final_final['DescRF_Modalidad'] = df_final_final.apply(lambda x: rec_modalidad(x.DescRF_Modalidad),axis=1)

In [26]:
# Manejo estrato
col = ['EE_ESTRATO_ACTUALIZADO']
df_desertores_final = df_final_final
for x in col:
    no_nulos = df_desertores_final[x].dropna()
    probs = no_nulos.value_counts(normalize=True)
    n_missing = df_desertores_final[x].isna().sum()
    imputados = np.random.choice(probs.index, size=n_missing, p=probs.values)
    df_desertores_final.loc[df_desertores_final[x].isna(), x] = imputados
df_final_final.EE_ESTRATO_ACTUALIZADO.value_counts(dropna=False,normalize=True)

ValueError: 'a' cannot be empty unless no samples are taken

In [27]:
# Estandarizar estado pago
def rec_estado_pago(x):
    if x == 'PAGO':
        return x 
    else:
        return 'NO'
df_final_final['ESTADO_PAGO'] = df_final_final.apply(lambda x: rec_estado_pago(x.ESTADO_PAGO),axis=1)

In [28]:
# Manejo DescRF_Acceso_Moodle a fecha 
df_final_final['Ultimo_Acc_Mood'] = pd.to_datetime(df_final_final['Ultimo_Acc_Mood'],errors='coerce')

fecha_actual = pd.to_datetime(datetime.now().date())
df_final_final['diferencia_dias_moodle_u'] = (fecha_actual - df_final_final['Ultimo_Acc_Mood']).dt.days

In [29]:
#Estado Moodle

df_final_final['NA_Calificaciones'] = df_final_final['NOT_PERIODO'].isna()
#df_final_final['EstadoMoodle'] = df_final_final.apply(lambda x: rec_estado(x.NA_Calificaciones,x.EstadoMoodle),axis=1)

def rec_estado(x,y):
    if x == False:
        return 'ACTIVO'
    else:
        return y
df_final_final['EstadoMoodle'] = df_final_final.apply(lambda x: rec_estado(x.NA_Calificaciones,x.EstadoMoodle),axis=1)

In [30]:
#Horarios
def rec_horario(x,y):
    if x == False:
        return 'SI'
    else:
        return 'NO'
df_final_final['Horario'] = df_final_final.apply(lambda x: rec_horario(x.NA_Calificaciones,x.Horario),axis=1)

In [31]:
#Correo 
def rec_correo(x):
    if x == 'SI':
        return x
    else:
        return 'NO'
df_final_final['Correo'] = df_final_final.apply(lambda x: rec_correo(x.Correo),axis=1)

In [32]:
#Recibo de Caja 
def rec_recibo_caja(x,y):
    if x == True:
        return 0
    else:
        return y   
df_final_final['NA_RECIBOS'] = df_final_final['RECIBOS_CAJA'].isna()
df_final_final['RECIBOS_CAJA'] = df_final_final.apply(lambda x: rec_recibo_caja(x.NA_RECIBOS,x.RECIBOS_CAJA),axis=1)

In [33]:
#Icetex 
df_final_final['NA_ICETEX'] = df_final_final['ICETEX'].isna()
df_final_final['ICETEX'] = df_final_final.apply(lambda x: rec_recibo_caja(x.NA_ICETEX,x.ICETEX),axis=1)

In [34]:
#Fincomercio 
df_final_final['NA_FINCOMERCIO'] = df_final_final['FINCOMERCIO'].isna()
df_final_final['FINCOMERCIO'] = df_final_final.apply(lambda x: rec_recibo_caja(x.NA_FINCOMERCIO,x.FINCOMERCIO),axis=1)

In [35]:
#Credity 
df_final_final['NA_CREDITY'] = df_final_final['CREDITY'].isna()
df_final_final['CREDITY'] = df_final_final.apply(lambda x: rec_recibo_caja(x.NA_CREDITY,x.CREDITY),axis=1)

In [36]:
#Total_Tickets
df_final_final['NA_Total_Tickets'] = df_final_final['Total_Tickets'].isna()
df_final_final['Total_Tickets'] = df_final_final.apply(lambda x: rec_recibo_caja(x.NA_Total_Tickets,x.Total_Tickets),axis=1)

In [37]:
df_final_final.columns

Index(['Identificacion', 'Periodo', 'DescRF_Status',
       'DescRF_Tipo_estado_alumno', 'DescRF_Modalidad', 'DescRF_Semestre_SINU',
       'DescRF_ciclo', 'DescRF_SEMESTRE_MEN', 'DescRF_Genero', 'DescRF_Unidad',
       'DescRF_Jornada', 'DescRF_Periodo_Ult_Pago', 'DescRF_Sede',
       'DescRF_Regional', 'DescRF_Programa', 'DescRF_Acceso_Moodle',
       'DescRF_tipo_estudio', 'Id_Alumn_Programa', 'DescRF_Tipo_Alumno',
       'DescRF_Nuevo', 'DescRF_orden', 'DescRF_Fecha_Liquidacion',
       'DescRF_Fecha_Nacimiento', 'DescRF_ultimo_curso',
       'DescAM_MATERIAS_INSCRITAS', 'DescAM_MATERIAS_APROBADAS',
       'DescAM_Porcentaje_aprobacion', 'EEpromedio', 'EE_UNIDADNEGOCIO',
       'EE_DEPARTAMENTO_REGIONAL', 'EE_OTRA_DISCAPACIDAD', 'EE_SISTEMA_SALUD',
       'EE_ESTRATO_ACTUALIZADO', 'EE_EMPRESA', 'COD_MATERIA', 'DEF_HISTORIA',
       'NOT_PERIODO', 'ESTADO_PAGO', 'EstadoMoodle', 'Ultimo_Acc_Mood',
       'Correo', 'Horario', 'VALOR_FINANCIACION', 'RECIBOS_CAJA', 'ICETEX',
       'FIN

In [39]:
import os
# -*- coding: utf-8 -*-
import pandas as pd
from sqlalchemy import create_engine, text

# ===============================================
# CONFIGURACIÓN DE CONEXIÓN
# ===============================================
# Ajusta tus credenciales y servidor
usuario = os.environ.get("SQL_USER","")
clave = os.environ.get("SQL_PASSWORD","")#$&/$*"    # cuidado con caracteres especiales
servidor = os.environ.get("SQL_SERVER","")
base_datos = os.environ.get("SQL_DB","")
schema = os.environ.get("SQL_SCHEMA","COE")
tabla = "df_final_GRU"

# Crear conexión con SQL Server (usa pyodbc)
connection_string = (
    f"mssql+pyodbc://{usuario}:{clave}@{servidor}/{base_datos}?driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(connection_string, fast_executemany=True)

# ===============================================
# CREAR TABLA (REEMPLAZAR SI YA EXISTE)
# ===============================================
with engine.begin() as conn:
    # Si la tabla ya existe, se elimina primero (opcional)
    conn.execute(text(f"IF OBJECT_ID('{schema}.{tabla}', 'U') IS NOT NULL DROP TABLE {schema}.{tabla};"))
    
    # Crear tabla desde el DataFrame
    df_final_final.to_sql(tabla, con=conn, schema=schema, index=False, if_exists='replace')
    print(f"✅ Tabla {schema}.{tabla} creada exitosamente en {base_datos}.")

# ===============================================
# VERIFICAR REGISTROS INSERTADOS
# ===============================================
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT COUNT(*) FROM {schema}.{tabla}"))
    total = result.scalar()
    print(f"📊 Registros insertados: {total}")


✅ Tabla COE.df_final_GRU creada exitosamente en CUN_REPOSITORIO.
📊 Registros insertados: 1750326


In [ ]:
df_final_final.head(3)

,Identificacion,Periodo,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_SEMESTRE_MEN,DescRF_Genero,DescRF_Unidad,...,Total_Tickets,Edad,Trabaja,diferencia_dias_moodle_u,NA_Calificaciones,NA_RECIBOS,NA_ICETEX,NA_FINCOMERCIO,NA_CREDITY,NA_Total_Tickets
0,00010105021,2017B,1,NUEVO,Presencial,1,TECNICO,2,M,TECNICA PROFESIONAL EN LOGISTICA DE COMERCIO E...,...,0.0,25,NO TRABAJA,NaN,False,True,True,True,True,True
1,00010105021,2017B,1,NUEVO,Presencial,1,TECNICO,2,M,TECNICA PROFESIONAL EN LOGISTICA DE COMERCIO E...,...,0.0,25,NO TRABAJA,NaN,False,True,True,True,True,True
2,00010105021,2017B,1,NUEVO,Presencial,1,TECNICO,2,M,TECNICA PROFESIONAL EN LOGISTICA DE COMERCIO E...,...,0.0,25,NO TRABAJA,NaN,False,True,True,True,True,True


In [ ]:
df_final_final.info()

In [ ]:
df_final_final.to_csv("base_fin_permanencia.csv.gz", index=False, compression='gzip')

In [ ]:
df_final_final = pd.read_csv(
    "C:/Users/maria_castrob/OneDrive - Corporación Unificada Nacional de Educación Superior - CUN/base_fin_permanencia.csv.gz",
    compression="gzip",
    sep=",",  
    engine="python"
)



### Creación base semana inicial

### Variables para modelo

In [ ]:
# Lista de variables seleccionadas
df_modelo = ['Identificacion','DescRF_Unidad','DescRF_Status',
    'DescRF_Modalidad', 'DescRF_ciclo', 'DescRF_SEMESTRE_MEN', 'DescRF_Genero',
    'DescRF_Unidad', 'DescRF_Jornada', 'DescRF_Sede', 'DescRF_Regional',
    'DescRF_Programa', 'DescRF_Acceso_Moodle', 'DescRF_tipo_estudio',
    'DescRF_Tipo_Alumno', 'DescRF_Nuevo', 'DescRF_orden',
    'DescAM_MATERIAS_INSCRITAS', 'DescAM_MATERIAS_APROBADAS',
    'DescAM_Porcentaje_aprobacion', 'EEpromedio', 'EE_ESTRATO_ACTUALIZADO',
    'COD_MATERIA', 'DEF_HISTORIA', 'NOT_PERIODO', 'ESTADO_PAGO', 'EstadoMoodle',
    'Correo', 'Horario', 'RECIBOS_CAJA', 'ICETEX',
    'FINCOMERCIO', 'CREDITY','Total_Tickets',
    'Edad', 'Trabaja'
]

# Crear la base del modelo a partir de df_final_final
base_modelo = df_final_final[df_modelo].copy()

# Verificar
print(base_modelo.shape)


In [ ]:
#base_modelo.to_csv("base_modelo_permanencia.csv.gz", index=False, compression='gzip')

In [ ]:
#base_modelo.head(3)

In [ ]:
#base_modelo.info()

In [ ]:
# Antes de todo
dups = base_modelo.columns[base_modelo.columns.duplicated()]
print("Columnas duplicadas:", dups)

# Nos quedamos solo con la primera aparición de cada nombre de columna
base_modelo = base_modelo.loc[:, ~base_modelo.columns.duplicated()]


Columnas duplicadas: Index(['DescRF_Unidad'], dtype='object')


In [ ]:
categoricas = base_modelo[variables_predictoras].select_dtypes(include=['object', 'category']).columns
base_modelo_encoded = pd.get_dummies(base_modelo, columns=categoricas, dummy_na=True)

# Por si acaso, también puedes asegurar que en el codificado no queden duplicadas:
base_modelo_encoded = base_modelo_encoded.loc[:, ~base_modelo_encoded.columns.duplicated()]


In [ ]:
# Columnas con alta cardinalidad identificadas como problemáticas:
columnas_alta_cardinalidad = [
    'DescRF_Unidad', 
    'DescRF_Programa', 
    'COD_MATERIA',
    'DescRF_Sede', 
    'DescRF_Regional'
]

# Definir un umbral de frecuencia. 
# Si un valor aparece menos de este número de veces, será agrupado como 'OTRO'.
# Para un dataset de 1.75 millones de filas, un umbral de 1000 a 5000 es un buen punto de partida.
UMBRAL_FRECUENCIA = 1000 

print(f"Reduciendo cardinalidad para: {columnas_alta_cardinalidad} con un umbral de {UMBRAL_FRECUENCIA} apariciones.")

for col in columnas_alta_cardinalidad:
    # 1. Calcular la frecuencia de cada valor
    frecuencias = base_modelo[col].value_counts()
    
    # 2. Identificar los valores raros (frecuencia menor al umbral)
    valores_raros = frecuencias[frecuencias < UMBRAL_FRECUENCIA].index
    
    # 3. Reemplazar los valores raros por 'OTRO'
    if len(valores_raros) > 0:
        base_modelo[col] = base_modelo[col].replace(valores_raros, 'OTRO')
        print(f"  Columna '{col}': {len(valores_raros)} categorías agrupadas en 'OTRO'. Nueva cardinalidad: {base_modelo[col].nunique()}")
    else:
        print(f"  Columna '{col}': No se encontraron categorías raras.")

# Convertir las columnas categóricas a tipo 'category' para optimizar la memoria antes del OHE
# Esto ayuda a que el siguiente paso (get_dummies) sea más eficiente
categoricas_optimizables = base_modelo.select_dtypes(include=['object']).columns
for col in categoricas_optimizables:
    base_modelo[col] = base_modelo[col].astype('category')

print("Proceso de reducción de cardinalidad y optimización de memoria completado.")


In [ ]:
# Target definido
target = 'DescRF_Status'
# Y que el identificador del estudiante esté claro
id_estudiante = 'Identificacion'

# 2. Definir las variables y en qué semana aparecen
# (AJUSTA ESTAS LISTAS SEGÚN TU CONOCIMIENTO DEL PROCESO)
semanas_totales = 16

vars_sem_1 = [
    'DescRF_Modalidad', 'DescRF_ciclo', 'DescRF_SEMESTRE_MEN', 'DescRF_Genero',
    'DescRF_Unidad', 'DescRF_Jornada', 'DescRF_Sede', 'DescRF_Regional',
    'DescRF_Programa', 'DescRF_tipo_estudio', 'DescRF_Tipo_Alumno',
    'DescRF_Nuevo', 'EE_ESTRATO_ACTUALIZADO', 'ESTADO_PAGO',
    'Correo', 'Horario', 'Edad', 'Trabaja'
]
vars_sem_3 = vars_sem_1 + ['DescRF_Acceso_Moodle', 'Total_Tickets']
vars_sem_8 = vars_sem_3 + [  'DescAM_MATERIAS_INSCRITAS']
vars_sem_12 = vars_sem_8 + [
    'RECIBOS_CAJA', 'ICETEX', 'FINCOMERCIO', 'CREDITY',
    'DescAM_MATERIAS_APROBADAS', 'DescAM_Porcentaje_aprobacion', 'EEpromedio',
    'DEF_HISTORIA', 'NOT_PERIODO'
]

# Lista con todas las variables predictoras
variables_predictoras = sorted(list(set(vars_sem_12)))

# 3. One-Hot Encoding para variables categóricas
# El modelo GRU necesita solo números.
categoricas = base_modelo[variables_predictoras].select_dtypes(include=['object', 'category']).columns
base_modelo_encoded = pd.get_dummies(base_modelo, columns=categoricas, dummy_na=True)

# Actualizar la lista de variables predictoras con las nuevas columnas dummies
variables_predictoras_encoded = [
    col for col in base_modelo_encoded.columns if any(var in col for var in variables_predictoras)
]


# 4. Crear el DataFrame secuencial
lista_dfs_semanales = []

for semana in range(1, semanas_totales + 1):
    df_semana = base_modelo_encoded[[id_estudiante, target]].copy()
    df_semana['semana'] = semana

    # Determinar qué variables están disponibles en esta semana
    vars_disponibles = []
    if semana >= 1:
        vars_disponibles.extend(vars_sem_1)
    if semana >= 3:
        vars_disponibles.extend(vars_sem_3)
    if semana >= 8:
        vars_disponibles.extend(vars_sem_8)
    if semana >= 12:
        vars_disponibles.extend(vars_sem_12)
    
    # Obtener las columnas dummy correspondientes a las vars disponibles
    cols_disponibles_encoded = [
        col for col in variables_predictoras_encoded if any(var in col for var in set(vars_disponibles))
    ]
    
    # Añadir solo las columnas disponibles para esta semana
    for col in variables_predictoras_encoded:
        if col in cols_disponibles_encoded:
            df_semana[col] = base_modelo_encoded[col]
        else:
            # np.nan es el valor que la capa Masking de Keras ignorará
            df_semana[col] = np.nan

    lista_dfs_semanales.append(df_semana)

# Unir todos los dataframes semanales en uno solo
df_secuencial = pd.concat(lista_dfs_semanales, ignore_index=True)

# 5. Guardar el archivo final listo para el modelo GRU
df_secuencial.to_csv("datos_secuenciales_para_gru.csv.gz", index=False, compression='gzip')

print("¡Archivo secuencial creado con éxito!")
print(df_secuencial.shape)
print(df_secuencial.head())
print(df_secuencial.tail())

In [ ]:
import pandas as pd
import numpy as np # <-- ¡Añadir esta línea si no está ya!

# ... (El código anterior: definiciones de target, id_estudiante, semanas, y vars_sem_X)

# 3. One-Hot Encoding para variables categóricas
# El modelo GRU necesita solo números.
categoricas = base_modelo[variables_predictoras].select_dtypes(include=['object', 'category']).columns
base_modelo_encoded = pd.get_dummies(base_modelo, columns=categoricas, dummy_na=True)

# *** CORRECCIÓN APLICADA AQUÍ ***
# Sanitizar nombres de columnas: Reemplazar espacios y caracteres especiales en los nombres de columna
# para evitar el error de slicing con nombres largos.
base_modelo_encoded.columns = base_modelo_encoded.columns.str.replace(' ', '_', regex=False)
base_modelo_encoded.columns = base_modelo_encoded.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Actualizar la lista de variables predictoras con las nuevas columnas dummies
# También se debe sanitizar la lista 'variables_predictoras' para que la condición 'var in col' funcione
variables_predictoras_sanitized = [var.replace(' ', '_').replace('[^A-Za-z0-9_]+', '', regex=True) 
                                   for var in variables_predictoras]

variables_predictoras_encoded = [
    col for col in base_modelo_encoded.columns 
    if any(var in col for var in variables_predictoras_sanitized)
]
# ***********************************

# 4. Crear el DataFrame secuencial
lista_dfs_semanales = []
for semana in range(1, semanas_totales + 1):
    df_semana = base_modelo_encoded[[id_estudiante, target]].copy()
    df_semana['semana'] = semana

    # Determinar qué variables están disponibles en esta semana
    vars_disponibles = []
    if semana >= 1:
        # Usar la lista no sanitizada para facilitar la definición inicial
        vars_disponibles.extend(vars_sem_1)
    if semana >= 3:
        # Aquí se añaden las nuevas variables que están solo en vars_sem_3
        vars_disponibles.extend([var for var in vars_sem_3 if var not in vars_sem_1])
    if semana >= 8:
        # Aquí se añaden las nuevas variables que están solo en vars_sem_8
        vars_disponibles.extend([var for var in vars_sem_8 if var not in vars_sem_3])
    if semana >= 12:
        # Aquí se añaden las nuevas variables que están solo en vars_sem_12
        vars_disponibles.extend([var for var in vars_sem_12 if var not in vars_sem_8])
    
    # Sanitizar las variables disponibles para que coincidan con los nombres de columna en base_modelo_encoded
    vars_disponibles_sanitized = [var.replace(' ', '_').replace('[^A-Za-z0-9_]+', '', regex=True) 
                                  for var in set(vars_disponibles)]
                                  
    # Obtener las columnas dummy correspondientes a las vars disponibles
    cols_disponibles_encoded = [
        col for col in variables_predictoras_encoded
        if any(var in col for var in vars_disponibles_sanitized)
    ]

    # Añadir solo las columnas disponibles para esta semana
    for col in variables_predictoras_encoded:
        if col in cols_disponibles_encoded:
            # Línea que fallaba: ahora 'col' es un nombre sanitizado y funciona
            df_semana[col] = base_modelo_encoded[col]
        else:
            # np.nan es el valor que la capa Masking de Keras ignorará
            df_semana[col] = np.nan

    lista_dfs_semanales.append(df_semana)

# Unir todos los dataframes semanales en uno solo
df_secuencial = pd.concat(lista_dfs_semanales, ignore_index=True)

# 5. Guardar el archivo final listo para el modelo GRU
df_secuencial.to_csv("datos_secuenciales_para_gru.csv.gz", index=False, compression='gzip')
print("¡Archivo secuencial creado con éxito!")
print(df_secuencial.shape)
print(df_secuencial.head())
print(df_secuencial.tail())

In [ ]:
base_modelo.to_csv("base_modelo_permanencia.csv", index=False, compression='gzip')